# Load Module dan Data

In [ ]:
import Textclassification as tc
import pandas as pd

In [ ]:
fSlang = 'data/slangword.txt'
bahasa = 'id'
stops = tc.LoadStopWords(bahasa)
sw=open(fSlang,encoding='utf-8', errors ='ignore', mode='r');SlangS=sw.readlines();sw.close()
SlangS = {slang.strip().split(':')[0]:slang.strip().split(':')[1] for slang in SlangS}

In [ ]:
data = pd.read_excel("C:\\Users\\Liza\\ANALISIS SENTIMENT\\data\\PROJECT_DATASENTIMENT.xlsx")

## Vektorisasi 

### Contoh Menggunakan data Tweet

In [ ]:
datatest = data.loc[0:1,:]

In [ ]:
datatest.Tweet

In [ ]:
cleantweet = []
for i in range(0, len(datatest.Tweet)):
    tweet = tc.cleanText(datatest.Tweet[i],fix = SlangS, lang = bahasa, stops = stops, lemmatizer= None,symbols_remove=True,min_charLen=3)
    print("Cleaning Tweets Indeks ke-", i, '\n', tweet, '\n')
    cleantweet.append(tweet)

## Pertama mari gunakan CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(cleantweet)

In [ ]:
print( type(tf) )
print('ukuran Matrix tf = {0}'.format(tf.shape))

In [ ]:
print(tf_vectorizer.vocabulary_)

In [ ]:
# kemudian ingat kembali bahwa tf_vectorizer.vocabulary_ bentuknya kata:index
# kita membutuhkan index:kata
# maka perlu di "inverse"
idx_kata = dict(map(reversed, tf_vectorizer.vocabulary_.items()))
print(idx_kata)

In [ ]:
kalimat_kedua = tf[1]
print(cleantweet[1],'\n')
index_kata_di_kalimat_kedua = kalimat_kedua.indices
print(index_kata_di_kalimat_kedua)

In [ ]:
kalimat_kedua = [idx_kata[idx] for idx in index_kata_di_kalimat_kedua]
print(kalimat_kedua)

In [ ]:
feature_names = tf_vectorizer.get_feature_names()
corpus_index = [n for n in cleantweet]
import pandas as pd
df = pd.DataFrame(tf.T.todense(), index=feature_names,columns=[corpus_index])
print(df)

In [ ]:
tf.toarray()

## Bagaimana dengan TFIDF ?

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(cleantweet)

In [ ]:
print( type(tfidf) )
print('ukuran Matrix tf = {0}'.format(tfidf.shape))

In [ ]:
tfidf_vectorizer.vocabulary_

In [ ]:
idx_katatfidf = dict(map(reversed, tfidf_vectorizer.vocabulary_.items()))
print(idx_katatfidf)

In [ ]:
kalimat_kedua = tf[1]
print(cleantweet[1],'\n')
index_kata_di_kalimat_kedua = kalimat_kedua.indices
print(index_kata_di_kalimat_kedua)

## Sejauh ini hasil tfidf dengan countvectorizer sama. Mari kita lanjutkan 

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()
corpus_index = [n for n in cleantweet]
import pandas as pd
df = pd.DataFrame(tfidf.T.todense(), index=feature_names,columns=[corpus_index])
print(df)

In [ ]:
tfidf.toarray()

# Word2Vec

In [ ]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

### Remove stop words

In [ ]:
def remove_stop_words(corpus):
    stop_words = ['is', 'a', 'will', 'be']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [ ]:
corpus = remove_stop_words(corpus)
print(corpus)

In [ ]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)

In [ ]:
words

# Data Generation

Kita akan mengenerate label/target dari setiap word menggunakan skip gram

In [ ]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []
for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])



In [ ]:
word2int

In [ ]:
sentences

In [ ]:
data

In [ ]:
import pandas as pd
for text in corpus:
    print(text)

In [ ]:
df = pd.DataFrame(data, columns = ['input', 'label'])
df

In [ ]:
df.shape

In [ ]:
word2int

# Define Tensorflow Graph

In [ ]:
import tensorflow as tf
import numpy as np

ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 2 

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

# Train

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 20000
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 3000 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

In [ ]:
# Now the hidden layer (W1 + b1) is actually the word look up table
vectors = sess.run(W1 + b1)
print(vectors)

# Word Cector in Table

In [ ]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

# Word Vector in 2d Chart

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for word, x1, x2 in zip(w2v_df['word'], w2v_df['x1'], w2v_df['x2']):
    ax.annotate(word, (x1,x2 ))
    
PADDING = 1.0
x_axis_min = np.amin(vectors, axis=0)[0] - PADDING
y_axis_min = np.amin(vectors, axis=0)[1] - PADDING
x_axis_max = np.amax(vectors, axis=0)[0] + PADDING
y_axis_max = np.amax(vectors, axis=0)[1] + PADDING
 
plt.xlim(x_axis_min,x_axis_max)
plt.ylim(y_axis_min,y_axis_max)
plt.rcParams["figure.figsize"] = (10,10)

plt.show()

# Sekarang mari kita coba Analisis Sentiment

In [ ]:
from textblob import TextBlob
# Lexicon Based berdasarkan pattern = https://www.clips.uantwerpen.be/pages/pattern-en#sentiment
Sentence = "Textblob is amazingly simple to use. What great fun!"
testimonial = TextBlob(Sentence)
print(testimonial.sentiment)
print('Polarity=Sentimen =', testimonial.sentiment.polarity)

In [ ]:
def moodyid(kalimat):
    K = TextBlob(kalimat).translate(to='en')
    pol,sub = K.sentiment
    if pol>0.05:
        pol='positive'
    else: 
        pol='negative'
    #else:
        #pol = 'netral'
    if sub>0.5:
        sub = 'Subjektif'
    else:
        sub = "Objektif"
    return pol, sub

In [ ]:
kalimat = 'anda itu sangat bodoh. hanya bisa mencontek'
moodyid(kalimat)

In [ ]:
TextBlob(kalimat).translate(to='en') #terbatas dengan api

In [ ]:
from textblob.sentiments import NaiveBayesAnalyzer
Sentence = "Textblob is amazingly simple to use. What great fun!"
blob = TextBlob(Sentence, analyzer=NaiveBayesAnalyzer())
blob.sentiment

# Sekarang bagaimana dengan data Twitter 

In [ ]:
datapos = pd.DataFrame(data[data.Label == 'Positive'])

In [ ]:
datapos = pd.DataFrame.reset_index(datapos,drop=True)
datapos = datapos.loc[0:5,]
datapos

In [ ]:
datapos.Tweet[2]

In [ ]:
dataneg = pd.DataFrame(data[data.Label == 'Negative'])

In [ ]:
dataneg = pd.DataFrame.reset_index(dataneg,drop=True)
dataneg = dataneg.loc[0:5,]
dataneg

In [ ]:
dataneg.Tweet[2]

In [ ]:
goldata = [datapos,dataneg]
goldata = pd.concat(goldata)
goldata = pd.DataFrame.reset_index(goldata, drop=True)
goldata

In [ ]:
goldata['Label'] = goldata['Label'].map({'Positive': 1, 'Negative': -1})

In [ ]:
goldata

In [ ]:
cleantweetbaru = []
for i in range(0, len(goldata.Tweet)):
    tweet = tc.cleanText(goldata.Tweet[i],fix = SlangS, lang = bahasa, stops = stops, lemmatizer= None,symbols_remove=True,min_charLen=3)
    print("Cleaning Tweets Indeks ke-", i, '\n', tweet, '\n')
    cleantweetbaru.append(tweet)

In [ ]:
cleantweetbaru = pd.DataFrame(cleantweetbaru)
cleantweetbaru = cleantweetbaru.rename(index=str, columns={0:'Tweet'})
cleantweetbaru

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(cleantweetbaru.Tweet)
print(tfidf.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(tfidf, goldata.Label, test_size=.3)
print(X_train.shape)

In [ ]:
print(Y_test)

In [ ]:
from sklearn import tree

DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train, Y_train)
prediksi = DT.predict(X_test)
print(prediksi)

In [ ]:
prediksiall = DT.predict(tfidf)
print(prediksiall,'\n')

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test,prediksi)

In [ ]:
from sklearn.metrics import accuracy_score
    
accuracy_score(Y_test, prediksi)

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import metrics

dSVM = svm.SVC(kernel='linear') # oneversus one SVM
model = dSVM.fit(X_train, Y_train)
print('Done!')

In [ ]:
prediksisvm = model.predict(X_test)
print(prediksisvm)

In [ ]:
prediksi = model.predict(tfidf)
print(prediksi,'\n')

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(goldata.Label, prediksi)

In [ ]:
from sklearn.metrics import accuracy_score
    
accuracy_score(goldata.Label, prediksi)

In [ ]:
# k-NN
from sklearn import neighbors

n_neighbors = 2
weights = 'distance'
kNN = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
kNN.fit(X_train, Y_train)
print('Done!')

In [ ]:
prediksi = kNN.predict(X_test)
accuracy_score(Y_test, prediksi)